In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime  #处理时间

In [2]:
user_cols = ['USER_ID','SHOP_ID','TIME_STA']
user_pay = pd.read_csv('./dataset/user_pay.txt',header=None, names = user_cols)
user_view = pd.read_csv('./dataset/user_view.txt',header=None,names = user_cols)
user_view_extra = pd.read_csv('./dataset/extra_user_view.txt',header=None,names = user_cols)
user_view = pd.concat([user_view, user_view_extra]) 

# 处理user_pay数据

In [3]:
user_pay.head()#2.3G

,USER_ID,SHOP_ID,TIME_STA
0,22127870,1862,2015-12-25 17:00:00
1,3434231,1862,2016-10-05 11:00:00
2,16955285,1862,2016-02-10 15:00:00
3,13799128,1862,2016-01-13 14:00:00
4,13799128,1862,2016-07-05 12:00:00


In [5]:
#将时间由string转为datetime
user_pay['TIME_STA'] = pd.to_datetime(user_pay['TIME_STA']) 
user_pay['DATE'] = pd.to_datetime(user_pay['TIME_STA']).dt.date
user_pay['HOUR'] = pd.to_datetime(user_pay['TIME_STA']).dt.hour

In [7]:
user_pay.head()

,USER_ID,SHOP_ID,TIME_STA,DATE,HOUR
0,22127870,1862,2015-12-25 17:00:00,2015-12-25,17
1,3434231,1862,2016-10-05 11:00:00,2016-10-05,11
2,16955285,1862,2016-02-10 15:00:00,2016-02-10,15
3,13799128,1862,2016-01-13 14:00:00,2016-01-13,14
4,13799128,1862,2016-07-05 12:00:00,2016-07-05,12


In [ ]:
#计算某个用户在一小时内对某店铺的下单次数  发现有异常值（某用户一小时内对某店铺下单200多次 刷单）
user_pay_new = user_pay.groupby(by =['USER_ID','SHOP_ID','DATE','HOUR'],as_index = False).count()

In [9]:
user_pay_new.head()

,USER_ID,SHOP_ID,DATE,HOUR,TIME_STA
0,1,25,2016-07-09,6,1
1,2,1848,2016-02-21,12,1
2,2,1848,2016-03-06,19,1
3,2,1848,2016-03-09,16,2
4,2,1848,2016-03-14,17,1


In [ ]:
user_pay_new = user_pay_new.rename(columns={'TIME_STA':'Num_raw'})

In [11]:
user_pay_new.head()

,USER_ID,SHOP_ID,DATE,HOUR,Num_raw
0,1,25,2016-07-09,6,1
1,2,1848,2016-02-21,12,1
2,2,1848,2016-03-06,19,1
3,2,1848,2016-03-09,16,2
4,2,1848,2016-03-14,17,1


In [12]:
#对单用户一小时内对店铺的下单次数做处理 X=1+log2X（2为底） 保证了x=1,2是处理之后还是1，2  x若是较大的异常值，处理之后会显著减小  
user_pay_new['Num_post'] = 1+ np.log(user_pay_new['Num_raw'])/ np.log(2) 

In [13]:
user_pay_new.head()

,USER_ID,SHOP_ID,DATE,HOUR,Num_raw,Num_post
0,1,25,2016-07-09,6,1,1.0
1,2,1848,2016-02-21,12,1,1.0
2,2,1848,2016-03-06,19,1,1.0
3,2,1848,2016-03-09,16,2,2.0
4,2,1848,2016-03-14,17,1,1.0


In [14]:
#计算某店铺一小时内的客流量
user_pay_new = user_pay_new.groupby(by =['SHOP_ID','DATE','HOUR'],as_index = False).sum()

In [15]:
user_pay_new.head()

,SHOP_ID,DATE,HOUR,USER_ID,Num_raw,Num_post
0,1,2015-10-10,15,110128409,14,14.0
1,1,2015-10-10,16,225244024,20,20.0
2,1,2015-10-10,17,186976200,16,16.0
3,1,2015-10-10,18,250975583,24,24.0
4,1,2015-10-10,19,493315216,48,48.0


In [16]:
#添加周几信息
user_pay_new['DofW'] = pd.to_datetime(user_pay_new['DATE']).dt.dayofweek

In [17]:
user_pay_new.head()

,SHOP_ID,DATE,HOUR,USER_ID,Num_raw,Num_post,DofW
0,1,2015-10-10,15,110128409,14,14.0,5
1,1,2015-10-10,16,225244024,20,20.0,5
2,1,2015-10-10,17,186976200,16,16.0,5
3,1,2015-10-10,18,250975583,24,24.0,5
4,1,2015-10-10,19,493315216,48,48.0,5


In [18]:
#因为不关注单个用户的情况，去掉了用户id
user_pay_new = user_pay_new.drop('USER_ID', 1)

In [19]:
user_pay_new.to_csv('user_pay_new.csv',index = False) #222.7MB

# 处理user_view数据 330M

In [25]:
user_view.head() 

,USER_ID,SHOP_ID,TIME_STA,DATE,HOUR
0,13201967,1197,2016-10-21 18:00:00,2016-10-21,18
1,19461365,1197,2016-06-28 23:00:00,2016-06-28,23
2,15022321,1197,2016-07-16 19:00:00,2016-07-16,19
3,5440872,1197,2016-07-15 07:00:00,2016-07-15,7
4,12594529,1197,2016-08-07 16:00:00,2016-08-07,16


In [20]:
#将时间由string转为datetime
user_view['TIME_STA'] = pd.to_datetime(user_view['TIME_STA'])
user_view['DATE'] = pd.to_datetime(user_view['TIME_STA']).dt.date
user_view['HOUR'] = pd.to_datetime(user_view['TIME_STA']).dt.hour

#跟user_pay做同样的操作
user_view_new = user_view.groupby(by =['USER_ID','SHOP_ID','DATE','HOUR'],as_index = False).count()
user_view_new = user_view_new.rename(columns={'TIME_STA':'Num_raw'})
user_view_new['Num_post'] = 1+ np.log(user_view_new['Num_raw'])/ np.log(2)
user_view_new = user_view_new.groupby(by =['SHOP_ID','DATE','HOUR'],as_index = False).sum()
user_view_new['DofW'] = pd.to_datetime(user_view_new['DATE']).dt.dayofweek
user_view_new = user_view_new.drop('USER_ID', 1)
user_view_new.to_csv('user_view_new.csv',index = False) #92M